# Remo Instance Segmentation Tutorial

## Install Dependencies : Detectron2

In [14]:
!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo
%reload_ext autoreload
%autoreload

In [2]:
# Imports
import os
import json

import remo
import numpy as np
from pycocotools import mask
from skimage import measure

import torch, torchvision

# Detectron 2 Imports
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances

## Dataset Configuration

In [ ]:
register_coco_instances("dog_train", {}, "./dog_train.json", "./images")
register_coco_instances("dog_test", {}, "./dog_test.json", "./images")
register_coco_instances("dog_valid", {}, "./dog_valid.json", "./images")

In [ ]:
segm_dogs_og_metadata = MetadataCatalog.get("dog_train")
dataset_dicts = DatasetCatalog.get("dog_train")

## Model Configuration

In [ ]:
model_yaml_path = "./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
model_weights_path = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"

In [ ]:

cfg = get_cfg()
cfg.merge_from_file(model_yaml_path)
cfg.DATASETS.TRAIN = ("dog_train",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_weights_path # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 3 classes (data, fig, hazelnut)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Inference

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("dog_test", )
predictor = DefaultPredictor(cfg)
test_dataset_dicts = DatasetCatalog.get("dog_test")

In [ ]:
for d in random.sample(test_dataset_dicts, 1):    
    outputs = predictor(im)

In [ ]:
pred_masks = outputs['instances'].get("pred_masks").cpu().permute(2, 1, 0).numpy()[:, :, 0]
polygons = Mask(pred_masks).polygons()
print(polygons.points)

## Uploading Predictions

In [ ]:
import json
import numpy as np
from pycocotools import mask
from skimage import measure

ground_truth_binary_mask =  tempmask.astype(np.uint8)

fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
ground_truth_area = mask.area(encoded_ground_truth)
ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
contours = measure.find_contours(ground_truth_binary_mask, 0.5)


annotation = {
        "segmentation": [],
        "area": ground_truth_area.tolist(),
        "iscrowd": 0,
        "image_id": 123,
        "bbox": ground_truth_bounding_box.tolist(),
        "category_id": 18,
        "id": 1
    }

annotations = []
for contour in contours:
    contour = np.flip(contour, axis=1)
    segmentation = contour.ravel().tolist()
    annotation["segmentation"].append([int(i) for i in segmentation])
print(json.dumps(annotation, indent=4))